In [ ]:
%%time
from src.data.make_dataset import load_list, get_stock_data
from src.models.StockModel import StockModel
window_size = 20
start_date = '2017-01-01'
end_date = '2024-09-01'
feature_columns = ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
target = "Open"

# Load symbols
nasdaq_symbols = load_list("NASDAQ")
sp500_symbols = load_list("SP500")

# Test tickers, sp500 symbols not also in nasdaq
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:25]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC']
train_tickers = ['^IXIC'] + nasdaq_symbols
train_tickers = train_tickers[:51]

# Download data
combined_data = get_stock_data(train_tickers, "1d", start_date, end_date)
combined_data.info()
# Test data
test_data = get_stock_data(test_tickers, "1d", start_date, end_date)

layer_config = [(1,128),(1,256),(1,512),(1,1024),(2,128),(2,256),(2,512),(2,768),(3,128),(3,256),(3,512)]
for i in layer_config:
    print(f"XXXXXXXXXXXXXXXX Running {i[0]} {i[1]} layers XXXXXXXXXXXXXXXXXXXX")
    # Create and train model
    stock_model = StockModel(window_size=window_size, feature_columns=feature_columns, target_name=target, export=True)
    
    stock_model.train(combined_data, patience=10, epochs=150, graph=False, layers=i[0], units_per_layer=i[1])
    metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=False)
    print(metrics_dict)
    print(mean_metrics)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94076 entries, 2017-01-03 00:00:00-05:00 to 2024-08-30 00:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          94076 non-null  float64
 1   High          94076 non-null  float64
 2   Low           94076 non-null  float64
 3   Close         94076 non-null  float64
 4   Volume        94076 non-null  int64  
 5   Dividends     94076 non-null  float64
 6   Stock Splits  94076 non-null  float64
 7   Ticker        94076 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 6.5+ MB
XXXXXXXXXXXXXXXX Running 1 128 layers XXXXXXXXXXXXXXXXXXXX
Initializing model:
 - Window size: 20
 - Features: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD']
 - Target: Open
--- Preparing ^IXIC data using 20 window---
Feature columns: ['Close', 'Open', 'Volume', 'Range', 'Gap', 'RSI', 'MACD', 'MACD_Hist']
Target column: Open
Feature min [ 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          70,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 70,273 (274.50 KB)

 Trainable params: 70,273 (274.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 256)                 │         271,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 271,617 (1.04 MB)

 Trainable params: 271,617 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

--- 1/51 Training model for ^IXIC ---
Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0095 - mape: 112.9677 - val_loss: 0.0084 - val_mape: 8.9664
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0334 - mape: 267.7789 - val_loss: 0.0080 - val_mape: 9.1406
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0124 - mape: 157.6374 - val_loss: 0.0021 - val_mape: 4.3968
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0028 - mape: 64.0469 - val_loss: 3.6204e-04 - val_mape: 1.8565
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 9.8257e-04 - mape: 33.8836 - val_loss: 3.1245e-04 - val_mape: 1.5932
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.4882e-04 - mape: 19.9347 - val_loss: 2.1192e-04 - val_mape: 1.3471
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 7.2342e-04 - mape: 30.1661 - val_loss: 2.1846e-04 - val_mape: 1.4255
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.6184e-04 - mape: 14.00

In [ ]:
test_tickers = [item for item in sp500_symbols if item not in nasdaq_symbols]
test_tickers = test_tickers[:5]

#tickers = ['^GSPC', '^IXIC', 'AAPL', 'MSFT', 'NVDA', 'AMZN', 'AVGO', 'META', 'GOOGL', 'GOOG', 'MMM', 'ADBE', 'BWA', 'GD', 'IT']
#tickers = ['^GSPC', '^IXIC', '^DJI']
# Test data
test_data = get_stock_data(["^GSPC", "^DJI"], "1d", start_date, end_date)

metrics_dict, mean_metrics = stock_model.evaluate_many(test_data, graph=True)
print(metrics_dict)
print(mean_metrics)
